# Exploratory Data Analysis: Отзывы Darling (Золотое Яблоко)

Цель: Исследовать датасет отзывов и подготовить данные для расчёта Loyalty Score

In [ ]:
# Импорты
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loader import load_data, get_data_info, print_data_summary
from src.preprocessing import clean_dataframe, get_text_statistics, extract_keywords

# Настройки отображения
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Для русского языка в графиках
plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. Загрузка данных

In [ ]:
# Загружаем данные
df = load_data('../data/data_darling.xlsx')
print(f"Загружено {len(df):,} записей")
df.head()

In [ ]:
# Сводка по данным
print_data_summary(df)

In [ ]:
# Типы данных и пропуски
print("\nТипы данных:")
print(df.dtypes)
print("\nПропуски:")
print(df.isnull().sum())

## 2. Анализ числовых переменных

In [ ]:
# Распределение Stars
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гистограмма
if 'stars' in df.columns:
    stars_counts = df['stars'].value_counts().sort_index()
    axes[0].bar(stars_counts.index.astype(str), stars_counts.values, color='gold', edgecolor='black')
    axes[0].set_xlabel('Stars')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Распределение оценок (Stars)')
    
    # Добавляем проценты
    for i, (idx, val) in enumerate(zip(stars_counts.index, stars_counts.values)):
        pct = val / len(df) * 100
        axes[0].text(i, val + 500, f'{pct:.1f}%', ha='center', fontsize=10)

# Распределение IsRecommended
if 'is_recommended' in df.columns:
    rec_counts = df['is_recommended'].value_counts()
    colors = ['#ff6b6b', '#51cf66']
    axes[1].pie(rec_counts.values, labels=['Не рекомендуют', 'Рекомендуют'], 
                autopct='%1.1f%%', colors=colors, explode=(0.05, 0))
    axes[1].set_title('Рекомендации')

plt.tight_layout()
plt.show()

In [ ]:
# Связь Stars и IsRecommended
if 'stars' in df.columns and 'is_recommended' in df.columns:
    cross_tab = pd.crosstab(df['stars'], df['is_recommended'], normalize='index') * 100
    print("Процент рекомендаций по оценкам:")
    print(cross_tab.round(1))
    
    # Визуализация
    cross_tab.plot(kind='bar', stacked=True, color=['#ff6b6b', '#51cf66'], figsize=(10, 5))
    plt.xlabel('Stars')
    plt.ylabel('Percent')
    plt.title('Рекомендации по оценкам')
    plt.legend(['Не рекомендуют', 'Рекомендуют'])
    plt.xticks(rotation=0)
    plt.show()

## 3. Анализ текстовых полей

In [ ]:
# Статистика по текстам
text_stats = get_text_statistics(df)

for field, stats in text_stats.items():
    print(f"\n{field.upper()}:")
    print(f"  Средняя длина: {stats['mean_length']:.0f} символов")
    print(f"  Медиана: {stats['median_length']:.0f} символов")
    print(f"  Макс: {stats['max_length']:.0f} символов")
    print(f"  Пустых: {stats['empty_percent']:.1f}%")

In [ ]:
# Примеры отзывов
print("=" * 60)
print("ПРИМЕРЫ ОТЗЫВОВ")
print("=" * 60)

sample = df.sample(5)
for idx, row in sample.iterrows():
    print(f"\n--- Отзыв #{idx} ---")
    print(f"Stars: {row.get('stars', 'N/A')} | Recommend: {row.get('is_recommended', 'N/A')}")
    print(f"Product: {row.get('product_name', 'N/A')[:50]}...")
    print(f"Pros: {str(row.get('pros', ''))[:150]}...")
    print(f"Cons: {str(row.get('cons', ''))[:150]}...")

In [ ]:
# Частотные слова в Pros
if 'pros' in df.columns:
    pros_keywords = extract_keywords(df['pros'], top_n=20)
    print("\nТоп-20 слов в PROS:")
    for word, count in pros_keywords.items():
        print(f"  {word}: {count}")

In [ ]:
# Частотные слова в Cons
if 'cons' in df.columns:
    cons_keywords = extract_keywords(df['cons'], top_n=20)
    print("\nТоп-20 слов в CONS:")
    for word, count in cons_keywords.items():
        print(f"  {word}: {count}")

## 4. Анализ категорий товаров

In [ ]:
# Топ категорий
if 'product_type' in df.columns:
    top_categories = df['product_type'].value_counts().head(15)
    
    plt.figure(figsize=(12, 6))
    top_categories.plot(kind='barh', color='steelblue')
    plt.xlabel('Количество отзывов')
    plt.title('Топ-15 категорий по количеству отзывов')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

In [ ]:
# Средняя оценка по категориям
if 'product_type' in df.columns and 'stars' in df.columns:
    category_stats = df.groupby('product_type').agg({
        'stars': ['mean', 'count'],
        'is_recommended': 'mean'
    }).round(2)
    category_stats.columns = ['avg_stars', 'count', 'recommend_rate']
    category_stats = category_stats[category_stats['count'] >= 100]  # Фильтр по количеству
    category_stats = category_stats.sort_values('avg_stars', ascending=False)
    
    print("\nТоп категорий по средней оценке (мин 100 отзывов):")
    print(category_stats.head(10))
    
    print("\nХудшие категории:")
    print(category_stats.tail(10))

## 5. Временной анализ

In [ ]:
# Динамика отзывов по времени
if 'created_date' in df.columns:
    df['date'] = pd.to_datetime(df['created_date']).dt.date
    
    daily_counts = df.groupby('date').size()
    
    plt.figure(figsize=(14, 5))
    daily_counts.plot(color='steelblue', alpha=0.7)
    plt.xlabel('Дата')
    plt.ylabel('Количество отзывов')
    plt.title('Динамика отзывов по дням')
    plt.tight_layout()
    plt.show()

In [ ]:
# Средняя оценка по месяцам
if 'created_date' in df.columns and 'stars' in df.columns:
    df['month'] = pd.to_datetime(df['created_date']).dt.to_period('M')
    
    monthly_stats = df.groupby('month').agg({
        'stars': 'mean',
        'is_recommended': 'mean'
    })
    
    fig, ax1 = plt.subplots(figsize=(14, 5))
    
    monthly_stats['stars'].plot(ax=ax1, color='gold', marker='o', label='Avg Stars')
    ax1.set_ylabel('Stars', color='gold')
    ax1.set_ylim(3.5, 5)
    
    ax2 = ax1.twinx()
    monthly_stats['is_recommended'].plot(ax=ax2, color='green', marker='s', label='Recommend Rate')
    ax2.set_ylabel('Recommend Rate', color='green')
    ax2.set_ylim(0.5, 1)
    
    plt.title('Динамика оценок и рекомендаций по месяцам')
    plt.tight_layout()
    plt.show()

## 6. Подготовка данных для Loyalty Score

In [ ]:
# Очистка данных
df_clean = clean_dataframe(df)
print(f"\nДобавлены признаки: {[c for c in df_clean.columns if c not in df.columns]}")

In [ ]:
# Распределение вовлечённости (заполненности полей)
if 'fields_filled' in df_clean.columns:
    filled_counts = df_clean['fields_filled'].value_counts().sort_index()
    
    plt.figure(figsize=(8, 5))
    filled_counts.plot(kind='bar', color='teal')
    plt.xlabel('Количество заполненных полей (Pros/Cons/Comment)')
    plt.ylabel('Количество отзывов')
    plt.title('Распределение вовлечённости в отзывы')
    plt.xticks(rotation=0)
    
    for i, val in enumerate(filled_counts.values):
        plt.text(i, val + 500, f'{val/len(df_clean)*100:.1f}%', ha='center')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Сохраняем очищенные данные
df_clean.to_pickle('../data/data_clean.pkl')
print("Очищенные данные сохранены в data/data_clean.pkl")

## Выводы EDA

**Ключевые наблюдения:**
1. TODO: Заполнить после анализа реальных данных
2. ...

**Следующие шаги:**
- Sentiment Analysis (notebook 02)
- Расчёт Loyalty Score
- Визуализация результатов